# Example with a linear difference system

Using the last update (dalgebra-0.0.20220428.1815) we can treat now difference rings too. Let's see how to work with them with a very simple linear system:

In [1]:
%display latex
from dalgebra import *

The system has 2 state variables $C(n)$ and $S(n)$ and the difference system can be written using the parameters $K_{se}, I_0, K_{cp}$ and $K_{rb}$. We need to create these variables and set the difference ring such that everything is a constant:

$$\begin{array}{rcl}
    I_0 & \mapsto & \texttt{I_0}\\
    K_{se} & \mapsto & \texttt{K_se}\\
    K_{cp} & \mapsto & \texttt{K_cp}\\
    K_{rb} & \mapsto & \texttt{K_rb}
\end{array}$$

In [2]:
R.<I_0, K_se, K_cp, K_rb, K_sc> = QQ[]
DR = DifferenceRing(R)
## We update the variables
I_0, K_se, K_cp, K_rb, K_sc = DR.gens()
## We check that all these are constants
print("All are constants -> ", all(el.difference() == el for el in DR.gens()))

DR

All are constants ->  True


Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In order to build the system, we need to create the two difference variables $C(n)$ and $S(n)$. In the code, these will be represented by $\texttt{C}$ and $\texttt{S}$ such that $\texttt{C[k]} = C(n+k)$ and $\texttt{S[k]} = S(n+k)$. We do that with the class `DifferencePolynomialRing`:

In [4]:
DPR.<C,S> = DifferencePolynomialRing(DR)
DPR

Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In [5]:
C, S

(C_*, S_*)

At this point we can create a Difference system with the appropiate equations:

In [6]:
from dalgebra.diff_polynomial.diff_polynomial_system import DifferenceSystem

system = DifferenceSystem([
    C[1] - K_se*I_0 * S[0] - (1 - K_cp - K_rb) * C[0] + K_se*S[0]*C[0],
    S[1] - (1 - K_se * I_0) * S[0] - K_rb * C[0] - K_sc * C[0] * S[0]
], variables = [C,S])
system

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

Let us consider now the output variables, that will be another system only focused on $S(n)$, so it will have coefficients using some shifts of $C(n)$:

In [7]:
A = [
    -C[1] + (1 - K_cp  - K_rb) * C[0],
    K_se * I_0 - K_se * C[0]
]
B = [
    K_rb * C[0],
    1 - K_se * I_0 + K_sc * C[0],
    -1
]

In [8]:
system_output = DifferenceSystem([
    A[0] + A[1] * S[0],
    B[0] + B[1] * S[0] + B[2] * S[1]
], variables = [S])
system_output

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	(-K_se)*C_0*S_0 - C_1 + (-K_cp - K_rb + 1)*C_0 + I_0*K_se*S_0 == 0
	K_sc*C_0*S_0 + K_rb*C_0 - S_1 + (-I_0*K_se + 1)*S_0 == 0
}

And what we have done by hand is that the system formed by $(f_1, \sigma f_1, f_2)$ is a nice system to delete $S(n)$:

In [9]:
extended_system = system_output.extend_by_difference([1,0])
extended_system.is_sp2()
extended_system

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	(-K_se)*C_0*S_0 - C_1 + (-K_cp - K_rb + 1)*C_0 + I_0*K_se*S_0 == 0
	(-K_se)*C_1*S_1 - C_2 + (-K_cp - K_rb + 1)*C_1 + I_0*K_se*S_1 == 0
	K_sc*C_0*S_0 + K_rb*C_0 - S_1 + (-I_0*K_se + 1)*S_0 == 0
}

In [10]:
extended_system.diff_resultant(alg_res="macaulay")

(K_se^2*K_rb - K_se*K_cp*K_sc - K_se*K_rb*K_sc + K_se*K_sc)*C_0^2*C_1 + (-K_se*K_sc)*C_0*C_1^2 + (-I_0*K_se^2*K_rb + I_0*K_se*K_cp*K_sc + I_0*K_se*K_rb*K_sc - I_0*K_se*K_sc)*C_0^2 + (I_0*K_se^2*K_cp - I_0*K_se^2 + I_0*K_se*K_sc)*C_0*C_1 + (I_0*K_se^2 - K_se)*C_1^2 + K_se*C_0*C_2 + (-I_0^2*K_se^2*K_cp + I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - I_0*K_se)*C_0 + (-I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + 2*I_0*K_se)*C_1 + (-I_0*K_se)*C_2

We can also try to compute this resultant from the original system (since `diff_resultant` extends the system):

In [11]:
system_output.diff_resultant(alg_res="macaulay")

(K_se^2*K_rb - K_se*K_cp*K_sc - K_se*K_rb*K_sc + K_se*K_sc)*C_0^2*C_1 + (-K_se*K_sc)*C_0*C_1^2 + (-I_0*K_se^2*K_rb + I_0*K_se*K_cp*K_sc + I_0*K_se*K_rb*K_sc - I_0*K_se*K_sc)*C_0^2 + (I_0*K_se^2*K_cp - I_0*K_se^2 + I_0*K_se*K_sc)*C_0*C_1 + (I_0*K_se^2 - K_se)*C_1^2 + K_se*C_0*C_2 + (-I_0^2*K_se^2*K_cp + I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - I_0*K_se)*C_0 + (-I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + 2*I_0*K_se)*C_1 + (-I_0*K_se)*C_2